In [ ]:
# Multi-Agent Demo: Sales Forecaster & Inventory Order Agents
#
# This notebook demonstrates three agent types:
# 1. sales_forecaster - Simple agent with system prompt, no tools
# 2. inventory_order - Agent with order_inventory tool
# 3. sales_and_order - Combined agent that creates child tasks for inventory ordering

from sb0 import Sb0

client = Sb0(base_url="http://localhost:5003")

In [ ]:
AGENT_NAME = "100-claude-agent-sdk"

# System prompts for different agent types
SALES_FORECASTER_PROMPT = """You are a Sales Forecasting Agent. Your role is to analyze sales data and provide forecasts.

When given historical sales data or asked about sales forecasts:
1. Analyze trends and patterns
2. Consider seasonal factors
3. Provide clear, actionable forecasts with confidence levels
4. Format your response with clear sections for different time periods

Always be specific with numbers and explain your reasoning."""

INVENTORY_ORDER_PROMPT = """You are an Inventory Order Agent. Your role is to help manage inventory orders.

You have access to a tool called 'order_inventory' that you can use to place inventory orders.
When a user requests to order inventory:
1. Confirm the month and number of units to order
2. Use the order_inventory tool to place the order
3. Report the result to the user

Always confirm order details before placing orders."""

In [ ]:
# Create a task with a specific agent type
# Available agent types:
# - "sales_forecaster" (default) - Simple sales forecasting agent
# - "inventory_order" - Inventory ordering with tool support
# - "sales_and_order" - Combined agent that creates child tasks
import uuid

# Change this to try different agent types
AGENT_TYPE = "sales_forecaster"  # or "inventory_order" or "sales_and_order"

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME, params={"name": f"{str(uuid.uuid4())[:8]}-task", "params": {"agent_type": AGENT_TYPE}}
)

task = rpc_response.result
print(f"Created task with agent_type='{AGENT_TYPE}'")
print(task)

In [5]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "create a file wihth hello world in it"},
        "task_id": task.id,
    },
)

event = rpc_response.result
print(event)

Event(id='d2f45967-f142-4194-986a-488604ea0a3e', agent_id='e4866b85-3d58-4f41-a8b5-6f79664e2b84', sequence_id=11, task_id='991abaac-f1fe-4058-ba45-717d26e8a0c8', content=TextContent(author='user', content='create a file wihth hello world in it', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 12, 15, 6, 9, 39, 751053, tzinfo=TzInfo(0)))


In [6]:
# Subscribe to the async task messages produced by the agent
from sb0.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task,
    only_after_timestamp=event.created_at,
    print_messages=True,
    rich_print=True,
    timeout=5,
)

╭───────────────────────── USER [12/15/2025 06:09:39] ─────────────────────────╮
│ create a file wihth hello world in it                                        │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [12/15/2025 06:09:43] ─────────────────────────╮
│ 🔧 Tool Request: Write                                                       │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "file_path": "/Users/oliver/code/anthropic/claude-code/hello.txt",        │
│    "content": "Hello World\n"                                                │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [12/15/2025 06:09:44] ─────────────────────────╮
│ 🔧 Tool Request: Write                                                       │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "file_path": "/Users/oliver/code/anthropic/claude-code/hello.txt",        │
│    "content": "Hello World\n"                                                │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────── USER [12/15/2025 06:09:44] ─────────────────────────╮
│ ✅ **Tool Response: **                                                       │
│                                                                              │
│ Claude requested permissions to write to                                     │
│ /Users/oliver/code/anthropic/claude-code/hello.txt, but you haven't granted  │
│ it yet.                                                                      │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [12/15/2025 06:09:46] ─────────────────────────╮
│ 🔧 Tool Request: Write                                                       │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "file_path": "/Users/oliver/code/anthropic/claude-code/hello.txt",        │
│    "content": "Hello World\n"                                                │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [12/15/2025 06:09:47] ─────────────────────────╮
│ 🔧 Tool Request: Write                                                       │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "file_path": "/Users/oliver/code/anthropic/claude-code/hello.txt",        │
│    "content": "Hello World\n"                                                │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────── USER [12/15/2025 06:09:47] ─────────────────────────╮
│ ✅ **Tool Response: **                                                       │
│                                                                              │
│ Claude requested permissions to write to                                     │
│ /Users/oliver/code/anthropic/claude-code/hello.txt, but you haven't granted  │
│ it yet.                                                                      │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [12/15/2025 06:09:50] ─────────────────────────╮
│ I need permission to create the file. Could you please grant write access so │
│ I can create hello.txt with "Hello World" in it?                             │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [12/15/2025 06:09:50] ─────────────────────────╮
│ DataContent: DataContent(author='agent', data={'subtype': 'success',         │
│ 'duration_ms': 9838, 'duration_api_ms': 18364, 'is_error': False,            │
│ 'num_turns': 3, 'session_id': '63c750a8-7167-43f6-b6a6-b210f8361401',        │
│ 'total_cost_usd': 0.043492750000000004, 'usage': {'input_tokens': 5,         │
│ 'cache_creation_input_tokens': 281, 'cache_read_input_tokens': 43915,        │
│ 'output_tokens': 210, 'server_tool_use': {'web_search_requests': 0,          │
│ 'web_fetch_requests': 0}, 'service_tier': 'standard', 'cache_creation':      │
│ {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 281}},         │
│ 'result': 'I need permission to create the file. Could you please grant      │
│ write access so I can create hello.txt with "Hello World" in it?',           │
│ 'structured_output': None}, style='static', type='data')                     │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 5 seconds - returning collected messages


In [6]:
import sys

print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"sys.path: {sys.path[:3]}")

Python executable: /Users/vraja/Desktop/cc-agentex/monorepo/.venv/bin/python
Python version: 3.12.11 (main, Sep 18 2025, 19:41:45) [Clang 20.1.4 ]
sys.path: ['/Users/vraja/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python312.zip', '/Users/vraja/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12', '/Users/vraja/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/lib-dynload']


In [10]:
import sb0

print(f"sb0 version: {sb0.__version__}")
print("Successfully imported sb0!")

ModuleNotFoundError: No module named 'sb0'